In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [3]:
!pip install catboost
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


In [5]:
#importing all required libraries
import pickle
import imblearn
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn import ensemble
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from catboost.core import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Data Loading

In [6]:
dataframe = pd.read_csv('/content/drive/MyDrive/nba_basketball_game_prediction/Dataset/2019_2024_nba.csv')
dataframe.head()

,Season,Rk,Team,Name,G,Date,At,Opp,W/L,Location,OPts,Opp_Name,DPts,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Win
0,2019,1,ATL,Atlanta Hawks,1,10/17/2018,@,NYK,L,Away,107,New York Knicks,126,93.8,110.5,114.0,0.200,0.400,0.546,49.5,46.3,6.1,4.5,0.511,19.7,16.3,0.167,0.515,12.4,79.2,0.242,0
1,2019,2,ATL,Atlanta Hawks,2,10/19/2018,@,MEM,L,Away,117,Memphis Grizzlies,131,109.9,123.1,106.5,0.434,0.361,0.592,49.4,65.9,8.5,9.1,0.578,16.1,24.4,0.253,0.623,14.5,80.6,0.370,0
2,2019,3,ATL,Atlanta Hawks,3,10/21/2018,@,CLE,W,Away,133,Cleveland Cavaliers,111,125.5,104.7,106.0,0.194,0.480,0.625,47.3,66.7,8.5,9.7,0.602,10.1,20.9,0.153,0.515,14.6,70.0,0.091,1
3,2019,4,ATL,Atlanta Hawks,4,10/24/2018,NaN,DAL,W,Home,111,Dallas Mavericks,104,102.7,96.2,108.1,0.341,0.418,0.530,50.9,75.7,5.5,11.8,0.489,12.5,18.9,0.242,0.484,14.2,83.0,0.176,1
4,2019,5,ATL,Atlanta Hawks,5,10/27/2018,NaN,CHI,L,Home,85,Chicago Bulls,97,82.3,94.0,103.2,0.274,0.476,0.452,48.0,74.1,8.7,16.0,0.393,18.9,20.0,0.226,0.506,20.3,82.2,0.157,0


In [7]:
dataframe.shape

(14118, 32)

In [8]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14118 entries, 0 to 14117
Data columns (total 32 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Season    14118 non-null  int64  
 1   Rk        14118 non-null  int64  
 2   Team      14118 non-null  object 
 3   Name      14118 non-null  object 
 4   G         14118 non-null  int64  
 5   Date      14118 non-null  object 
 6   At        7059 non-null   object 
 7   Opp       14118 non-null  object 
 8   W/L       14118 non-null  object 
 9   Location  14118 non-null  object 
 10  OPts      14118 non-null  int64  
 11  Opp_Name  14118 non-null  object 
 12  DPts      14118 non-null  int64  
 13  ORtg      14118 non-null  float64
 14  DRtg      14118 non-null  float64
 15  Pace      14118 non-null  float64
 16  FTr       14118 non-null  float64
 17  3PAr      14118 non-null  float64
 18  TS%       14118 non-null  float64
 19  TRB%      14118 non-null  float64
 20  AST%      14118 non-null  fl

In [ ]:
dataframe.columns

Index(['Season', 'Rk', 'Team', 'Name', 'G', 'Date', 'At', 'Opp', 'W/L',
       'Location', 'OPts', 'Opp_Name', 'DPts', 'ORtg', 'DRtg', 'Pace', 'FTr',
       '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%',
       'FT/FGA', 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1', 'Win'],
      dtype='object')

# Data Cleaning

In [ ]:
#dropping  unnecessary columns
dataframe = dataframe.drop(['Rk', 'G', 'At', 'Win'], axis='columns')

In [ ]:
#convert datatype from object to datetime format
dataframe['Date'] = pd.to_datetime(dataframe['Date'])

In [ ]:
#renaming column name
dataframe.rename(columns = {
    "OPts" : "Points",
    "DPts" : "Opponent Points",
    "ORtg" : "Offensive Rating",
    "DRtg" : "Defensive Rating",
    "FTr" : "Free Throw Attempt Rate",
    "3PAr" : "3 Point Attempt Rate",
    "TS%" : "True Shooting Percentage",
    "TRB%" : "Total Rebound Percentage",
    "AST%" : "Assist Percentage",
    "STL%" : "Steal Percentage",
    "BLK%" : "Block Percentage",
    "eFG%" : "Effective Field Goal Percentage",
    "TOV%" : "Turnover Percentage",
    "ORB%" : "Offensive Rebound Percentage",
    "FT/FGA" : "Free Throws Per Field Goal Attempt",
    "eFG%.1" : "Opponent Effective Field Goal Percentage",
    "TOV%.1" : "Opponent Turnover Percentage",
    "DRB%" : "Defensive Rebound Percentage",
    "FT/FGA.1" : "Opponent Free Throws Per Field Goal Attempt",
    "Opp" : "Opponent Team"

    }, inplace = True)
dataframe.head()

,Season,Team,Name,Date,Opponent Team,W/L,Location,Points,Opp_Name,Opponent Points,Offensive Rating,Defensive Rating,Pace,Free Throw Attempt Rate,3 Point Attempt Rate,True Shooting Percentage,Total Rebound Percentage,Assist Percentage,Steal Percentage,Block Percentage,Effective Field Goal Percentage,Turnover Percentage,Offensive Rebound Percentage,Free Throws Per Field Goal Attempt,Opponent Effective Field Goal Percentage,Opponent Turnover Percentage,Defensive Rebound Percentage,Opponent Free Throws Per Field Goal Attempt
0,2019,ATL,Atlanta Hawks,2018-10-17,NYK,L,Away,107,New York Knicks,126,93.8,110.5,114.0,0.200,0.400,0.546,49.5,46.3,6.1,4.5,0.511,19.7,16.3,0.167,0.515,12.4,79.2,0.242
1,2019,ATL,Atlanta Hawks,2018-10-19,MEM,L,Away,117,Memphis Grizzlies,131,109.9,123.1,106.5,0.434,0.361,0.592,49.4,65.9,8.5,9.1,0.578,16.1,24.4,0.253,0.623,14.5,80.6,0.370
2,2019,ATL,Atlanta Hawks,2018-10-21,CLE,W,Away,133,Cleveland Cavaliers,111,125.5,104.7,106.0,0.194,0.480,0.625,47.3,66.7,8.5,9.7,0.602,10.1,20.9,0.153,0.515,14.6,70.0,0.091
3,2019,ATL,Atlanta Hawks,2018-10-24,DAL,W,Home,111,Dallas Mavericks,104,102.7,96.2,108.1,0.341,0.418,0.530,50.9,75.7,5.5,11.8,0.489,12.5,18.9,0.242,0.484,14.2,83.0,0.176
4,2019,ATL,Atlanta Hawks,2018-10-27,CHI,L,Home,85,Chicago Bulls,97,82.3,94.0,103.2,0.274,0.476,0.452,48.0,74.1,8.7,16.0,0.393,18.9,20.0,0.226,0.506,20.3,82.2,0.157


In [ ]:
#dropping duplicates
dataframe.drop_duplicates(inplace=True)
dataframe.shape

(14118, 28)

In [ ]:
#checking null values
dataframe.isna().sum()

,0
Season,0
Team,0
Name,0
Date,0
Opponent Team,0
W/L,0
Location,0
Points,0
Opp_Name,0
Opponent Points,0


In [ ]:
dataframe.describe()

,Season,Date,Points,Opponent Points,Offensive Rating,Defensive Rating,Pace,Free Throw Attempt Rate,3 Point Attempt Rate,True Shooting Percentage,Total Rebound Percentage,Assist Percentage,Steal Percentage,Block Percentage,Effective Field Goal Percentage,Turnover Percentage,Offensive Rebound Percentage,Free Throws Per Field Goal Attempt,Opponent Effective Field Goal Percentage,Opponent Turnover Percentage,Defensive Rebound Percentage,Opponent Free Throws Per Field Goal Attempt
count,14118.000000,14118,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000
mean,2021.546961,2021-08-09 17:21:11.398215168,112.457855,112.457855,112.695247,112.695247,99.205737,0.257111,0.386631,0.571861,50.000935,60.703527,7.535451,8.970931,0.537096,12.417318,23.036429,0.199672,0.537096,12.417318,76.964513,0.199672
min,2019.000000,2018-10-16 00:00:00,68.000000,68.000000,67.800000,67.800000,82.700000,0.000000,0.109000,0.356000,31.900000,27.000000,0.000000,0.000000,0.301000,1.000000,0.000000,0.000000,0.301000,1.000000,44.200000,0.000000
25%,2020.000000,2020-01-05 00:00:00,104.000000,104.000000,104.800000,104.800000,96.000000,0.195000,0.333000,0.530000,46.400000,54.300000,5.500000,5.800000,0.490000,10.000000,17.900000,0.147000,0.490000,10.000000,72.100000,0.147000
50%,2022.000000,2021-11-10 00:00:00,112.000000,112.000000,112.700000,112.700000,99.100000,0.250000,0.384500,0.570000,50.000000,61.000000,7.300000,8.600000,0.535000,12.300000,22.900000,0.193000,0.535000,12.300000,77.100000,0.193000
75%,2023.000000,2023-01-21 00:00:00,121.000000,121.000000,120.600000,120.600000,102.300000,0.312000,0.437000,0.614000,53.600000,67.400000,9.300000,11.800000,0.580000,14.600000,27.900000,0.244000,0.580000,14.600000,82.100000,0.244000
max,2024.000000,2024-04-14 00:00:00,176.000000,176.000000,158.400000,158.400000,118.700000,0.739000,0.692000,0.808000,68.100000,91.900000,21.100000,40.400000,0.795000,29.300000,55.800000,0.638000,0.795000,29.300000,100.000000,0.638000
std,1.728435,NaN,12.600808,12.600808,11.683723,11.683723,4.730502,0.088501,0.076349,0.061650,5.352692,9.506219,2.815710,4.426725,0.066019,3.384308,7.237438,0.073206,0.066019,3.384308,7.237482,0.073206


# Data Visualization (Exploratory Data Analysis)

In [ ]:
#for analysis
eda_dataframe = dataframe

In [ ]:
eda1 = eda_dataframe[['Season', 'Team', 'W/L']].groupby(['Season', 'Team']).value_counts().reset_index(name='count')

fig = px.bar(eda1, x='Team', y='count', color='W/L', barmode='group', facet_col='Season', facet_col_wrap=2,
                title="Team Performance by Season and Outcome \n", labels={'count': 'Count', 'Team': 'Team', 'W/L': 'Win/Loss'})

fig.update_layout(height=600, width=1100, margin=dict(l=20, r=20, t=40, b=20))
fig.show()

In [ ]:
eda2 = eda_dataframe[['Location', 'W/L']].groupby(['Location', 'W/L']).value_counts().reset_index(name='count')
eda2['Location_W/L'] = eda2['Location']+'_'+eda2['W/L']
eda2 = eda2[['Location_W/L', 'count']]
fig = px.pie(eda2, values='count', names='Location_W/L', hole=0.4, title="Location Win/Loss Distribution", color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
eda3 = eda_dataframe[['Points']]
fig = px.histogram(eda3, x='Points', title="Distribution of Points", nbins=20, marginal="box", color_discrete_sequence=['yellowgreen'])
fig.update_layout(xaxis_title="Overall Points", yaxis_title="Games")
fig.show()

In [ ]:
eda4 = eda_dataframe[['Date', 'Points']]
fig = px.scatter(eda4, x='Date', y='Points', color=eda4['Date'].dt.year, title="Points vs Date")
fig.show()

In [ ]:
eda5 = eda_dataframe[['True Shooting Percentage', 'Team']].groupby('Team').mean().reset_index()
fig = px.line(eda5, y='True Shooting Percentage', x='Team', title='Avg True Shooting Percentage by Team', markers=True)
fig.show()

In [ ]:
fig = px.scatter(eda_dataframe, x='Points', y='Opponent Points', color='Team',
                 title="Points Scored vs Opponent Points",
                 labels={'Points': 'Points Scored', 'Opponent Points': 'Opponent Points'})
fig.show()

In [ ]:
eda_dataframe['Point Difference'] = eda_dataframe['Points'] - eda_dataframe['Opponent Points']
win_df = eda_dataframe[eda_dataframe['W/L'] == 'W']
biggest_comebacks = win_df.sort_values(by='Point Difference', ascending=False)
top_10_comebacks = biggest_comebacks[['Date', 'Team', 'Opp_Name', 'Points', 'Opponent Points', 'Point Difference']].head(10)

fig = px.bar(top_10_comebacks, x='Date', y='Point Difference', color='Team',
             title='Top 10 Biggest Comebacks in NBA Games',
             labels={'Point Difference': 'Point Difference (Biggest Comeback)', 'Date': 'Game Date'},
             text='Opp_Name')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.show()


In [ ]:
eda6 = eda_dataframe.groupby(['Season', 'Team'])['Turnover Percentage'].mean().reset_index()
fig = px.bar(eda6, x='Season', y='Turnover Percentage', color='Team', title='Team Avg Turnover Percentage Over Seasons', text_auto=True)
fig.show()

In [ ]:
fig = px.scatter(eda_dataframe, x='Turnover Percentage', y='Points', color='Team', hover_name='Name', title='Player Turnover Percentage vs Points',trendline="ols")
fig.show()

In [ ]:
eda7 = eda_dataframe.groupby(['Season', 'Team'])['Steal Percentage'].mean().reset_index()
fig = px.bar(eda7,x='Season',y='Steal Percentage',color='Team',title='Team Avg Steal Percentage Over Seasons', barmode='group')
fig.show()

In [ ]:
fig = px.box(eda_dataframe, x='W/L', y='Effective Field Goal Percentage', color='W/L', title='Effective Field Goal Percentage in Wins vs Losses')
fig.show()

In [ ]:
eda8 = eda_dataframe[['W/L']].groupby(['W/L']).value_counts().reset_index(name='count')
fig = px.pie(eda8, values='count', names='W/L', title="Count Plot of W/L")
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
eda9 = eda_dataframe[['Season', 'Team', 'Defensive Rating']].groupby(['Season', 'Team']).mean().reset_index()

fig = px.scatter(eda9, x='Team', y='Defensive Rating', size='Defensive Rating', color='Team', facet_col='Season', facet_col_wrap=2,
                title="Season Wise Avg Defensive Rating by Team \n", labels={'Defensive Rating': 'Avg Defensive Rating', 'Team': 'Team', 'Season': 'Season'})

fig.update_layout(height=600, width=1100, margin=dict(l=20, r=20, t=40, b=20))
fig.show()

In [ ]:
eda10 = eda_dataframe[['Season', 'Team', 'True Shooting Percentage']].groupby(['Season', 'Team']).mean().reset_index()

fig = px.line(eda10, x='Team', y='True Shooting Percentage', facet_col='Season', facet_col_wrap=2, color_discrete_sequence=px.colors.sequential.Sunsetdark_r, markers=True,
                title="Season Wise Avg True Shooting Percentage by Team \n", labels={'True Shooting Percentage': 'Avg True Shooting Percentage', 'Team': 'Team', 'Season': 'Season'})

fig.update_layout(height=600, width=1100, margin=dict(l=20, r=20, t=40, b=20))
fig.show()

In [ ]:
corr_matrix = eda_dataframe.corr(numeric_only=True).round(2)
fig = px.imshow(corr_matrix, text_auto=True, aspect="auto", title="Correlation Matrix")
fig.update_layout(width = 1400, height = 800, autosize = False)
fig.show()

# Data Preprocessing

In [ ]:
#dropping unnecessary columns
dataframe.drop(['Name', 'Opp_Name', 'Date', 'Point Difference'], axis='columns', inplace=True)
dataframe.head()

,Season,Team,Opponent Team,W/L,Location,Points,Opponent Points,Offensive Rating,Defensive Rating,Pace,Free Throw Attempt Rate,3 Point Attempt Rate,True Shooting Percentage,Total Rebound Percentage,Assist Percentage,Steal Percentage,Block Percentage,Effective Field Goal Percentage,Turnover Percentage,Offensive Rebound Percentage,Free Throws Per Field Goal Attempt,Opponent Effective Field Goal Percentage,Opponent Turnover Percentage,Defensive Rebound Percentage,Opponent Free Throws Per Field Goal Attempt
0,2019,ATL,NYK,L,Away,107,126,93.8,110.5,114.0,0.200,0.400,0.546,49.5,46.3,6.1,4.5,0.511,19.7,16.3,0.167,0.515,12.4,79.2,0.242
1,2019,ATL,MEM,L,Away,117,131,109.9,123.1,106.5,0.434,0.361,0.592,49.4,65.9,8.5,9.1,0.578,16.1,24.4,0.253,0.623,14.5,80.6,0.370
2,2019,ATL,CLE,W,Away,133,111,125.5,104.7,106.0,0.194,0.480,0.625,47.3,66.7,8.5,9.7,0.602,10.1,20.9,0.153,0.515,14.6,70.0,0.091
3,2019,ATL,DAL,W,Home,111,104,102.7,96.2,108.1,0.341,0.418,0.530,50.9,75.7,5.5,11.8,0.489,12.5,18.9,0.242,0.484,14.2,83.0,0.176
4,2019,ATL,CHI,L,Home,85,97,82.3,94.0,103.2,0.274,0.476,0.452,48.0,74.1,8.7,16.0,0.393,18.9,20.0,0.226,0.506,20.3,82.2,0.157


In [ ]:
#splitting dataset into feature and target column
X = dataframe.drop('W/L', axis='columns')
y = dataframe['W/L']
y = y.map({'W': 1, 'L': 0})

In [ ]:
catcol = X.select_dtypes(include='object').columns
catcol

Index(['Team', 'Opponent Team', 'Location'], dtype='object')

In [ ]:
#labelencoder (converting categorical data to numerical)
le = LabelEncoder()
for col in catcol:
    X[col] = le.fit_transform(X[col])
X.head()

,Season,Team,Opponent Team,Location,Points,Opponent Points,Offensive Rating,Defensive Rating,Pace,Free Throw Attempt Rate,3 Point Attempt Rate,True Shooting Percentage,Total Rebound Percentage,Assist Percentage,Steal Percentage,Block Percentage,Effective Field Goal Percentage,Turnover Percentage,Offensive Rebound Percentage,Free Throws Per Field Goal Attempt,Opponent Effective Field Goal Percentage,Opponent Turnover Percentage,Defensive Rebound Percentage,Opponent Free Throws Per Field Goal Attempt
0,2019,0,19,0,107,126,93.8,110.5,114.0,0.200,0.400,0.546,49.5,46.3,6.1,4.5,0.511,19.7,16.3,0.167,0.515,12.4,79.2,0.242
1,2019,0,14,0,117,131,109.9,123.1,106.5,0.434,0.361,0.592,49.4,65.9,8.5,9.1,0.578,16.1,24.4,0.253,0.623,14.5,80.6,0.370
2,2019,0,5,0,133,111,125.5,104.7,106.0,0.194,0.480,0.625,47.3,66.7,8.5,9.7,0.602,10.1,20.9,0.153,0.515,14.6,70.0,0.091
3,2019,0,6,1,111,104,102.7,96.2,108.1,0.341,0.418,0.530,50.9,75.7,5.5,11.8,0.489,12.5,18.9,0.242,0.484,14.2,83.0,0.176
4,2019,0,3,1,85,97,82.3,94.0,103.2,0.274,0.476,0.452,48.0,74.1,8.7,16.0,0.393,18.9,20.0,0.226,0.506,20.3,82.2,0.157


In [ ]:
# splitting the data into train and test with ratio of 90:10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [ ]:
X_train.shape

(12706, 24)

In [ ]:
feature_name = X.columns.values
model = ensemble.ExtraTreesClassifier()
model.fit(X,y)

#plot imp
importance = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],axis=0)
indices = np.argsort(importance)[::-1][:]

#graph
fig = go.Figure()
fig.add_trace(go.Bar(x=[feature_name[i] for i in indices], y=[importance[i] for i in indices], error_y=dict(type='data',array=[std[i] for i in indices],
            visible=True),marker=dict(color='yellowgreen'),name="Feature Importances"))
fig.update_layout(title="Feature Importances",xaxis=dict(title="Features", tickangle=90),yaxis=dict(title="Importance"),bargap=0.2,  autosize=False,width=1300,height=900)
fig.show()

# Machine Learning Models

#Logistic Regression

In [ ]:
#model
lgrclf = LogisticRegression(max_iter=2)
lgrclf.fit(X_train,y_train)
y_pred = lgrclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.8937677053824362


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88       706
           1       0.82      1.00      0.90       706

    accuracy                           0.89      1412
   macro avg       0.91      0.89      0.89      1412
weighted avg       0.91      0.89      0.89      1412



# Decision Tree Classifier

In [ ]:
#model
dtclf = DecisionTreeClassifier(max_depth=2)
dtclf.fit(X_train,y_train)
y_pred = dtclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.8618980169971672


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       706
           1       0.88      0.84      0.86       706

    accuracy                           0.86      1412
   macro avg       0.86      0.86      0.86      1412
weighted avg       0.86      0.86      0.86      1412



# Random Forest Classifier

In [ ]:
#model
rfclf = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=3)
rfclf.fit(X_train,y_train)
y_pred = rfclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.9256373937677054


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       706
           1       0.94      0.91      0.92       706

    accuracy                           0.93      1412
   macro avg       0.93      0.93      0.93      1412
weighted avg       0.93      0.93      0.93      1412



# AdaBoost Classifier

In [ ]:
#model
adbclf = AdaBoostClassifier(n_estimators=10, algorithm="SAMME")
adbclf.fit(X_train,y_train)
y_pred = adbclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.9270538243626062


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.92       706
           1       0.90      0.96      0.93       706

    accuracy                           0.93      1412
   macro avg       0.93      0.93      0.93      1412
weighted avg       0.93      0.93      0.93      1412



# CatBoost Classifier

In [ ]:
#model
cbclf = CatBoostClassifier(n_estimators=6, max_depth=2, verbose=0)
cbclf.fit(X_train,y_train)
y_pred = cbclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.9504249291784702


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95       706
           1       0.93      0.97      0.95       706

    accuracy                           0.95      1412
   macro avg       0.95      0.95      0.95      1412
weighted avg       0.95      0.95      0.95      1412



# LGBM Classifier

In [ ]:
#model
lgbmclf = LGBMClassifier(n_estimators=8, boosting_type='gbdt', max_depth=2, verbose=-1)
lgbmclf.fit(X_train,y_train)
y_pred = lgbmclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.9313031161473088


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       706
           1       0.95      0.91      0.93       706

    accuracy                           0.93      1412
   macro avg       0.93      0.93      0.93      1412
weighted avg       0.93      0.93      0.93      1412



# Hybrid Ensembled Classifier

In [ ]:
hybridensclf = StackingClassifier(classifiers=[LGBMClassifier(n_estimators=8, max_depth=6, verbose=-1),CatBoostClassifier(n_estimators=6, max_depth=6, verbose=0), RandomForestClassifier(n_estimators=6)],meta_classifier=AdaBoostClassifier(n_estimators=5))
hybridensclf.fit(X_train,y_train)
y_pred = hybridensclf.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test,y_pred))

Accuracy Score:  0.9808781869688386


In [ ]:
#confusion matrix
matrix =confusion_matrix(y_test, y_pred)
fig = px.imshow(matrix, text_auto=True, aspect="auto", title="Confusion Matrix")
fig.update_layout(width = 400, height = 400, autosize = False)
fig.show()

In [ ]:
#Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       706
           1       0.98      0.99      0.98       706

    accuracy                           0.98      1412
   macro avg       0.98      0.98      0.98      1412
weighted avg       0.98      0.98      0.98      1412

